In [ ]:
import json
import re

In [ ]:
import opik

# Opik 클라이언트 초기화
client = opik.Opik()

rest_client = client.rest_client

In [ ]:
traces = rest_client.traces.get_traces_by_project(project_name="Langflow", size=98) # Project에서 Trace 추출

### 질문 추출

In [ ]:
# 질문
print(traces.content[0].input['Chat Input (ChatInput-pCu53)']['input_value']) # base

### 답변 추출

In [ ]:
# 답변
print(traces.content[10].output["Chat Output (ChatOutput-Xv0g3)"]["message"]["text"]) # base

### context 추출

In [ ]:
# context
print(traces.content[10].output["Pure Vector Retriever (audit_smart_sql_pgvector-zNCxe)"]["retriever_output"][0]['data']['text']) # base

### 평가 시 데이터 양식
```
sample = {
    "question": "직원 A의 행위는 위법한가?",
    "answer": "직원 A의 행위는 청탁금지법 위반이다.",
    "contexts": [
        "직원 A는 승진 심사 기간 중 심사위원에게 식사를 제공하였다.",
        "청탁금지법 제8조는 직무관련자에게 금품 제공을 금지하고 있다."
    ],
}
```

## Ragas 평가지표 계산

In [ ]:
# 평가 준비
evals = []

In [ ]:
# Base
for trace in traces.content[::-1]:
    row = dict()
    row['question'] = trace.input["Chat Input (ChatInput-pCu53)"]['input_value']
    row['answer'] = trace.output["Chat Output (ChatOutput-Xv0g3)"]["message"]["text"]
    context = trace.output["Pure Vector Retriever (audit_smart_sql_pgvector-zNCxe)"]["retriever_output"]
    row['contexts'] = [c['data']['text'] for c in context]
    evals.append(row) # 평가 데이터셋에 추가

### 평가 시작

In [ ]:
import pandas as pd
from datasets import Dataset
from ragas import evaluate
from ragas.metrics import faithfulness, answer_relevancy
from langchain_openai import ChatOpenAI

In [ ]:
# results 형변환
df = pd.DataFrame(evals)
ragas_dataset = Dataset.from_pandas(df)

In [ ]:
# ragas 최신버전은 Langchain LLM 객체 형태를 받는 구조로 설계됨
judge_llm = ChatOpenAI(model='gpt-4.1-mini', temperature=0) # 4.1 mini가 가격, 성능 대비 가장 최적, 없는 모델이면 invoke시 오류 발생하여 테스트 가능

faithfulness.llm = judge_llm
answer_relevancy.llm = judge_llm

In [ ]:
metrics = [
    faithfulness,
    answer_relevancy,
]

In [ ]:
score = evaluate(ragas_dataset, metrics=metrics)

## Opik에 trace 생성

In [ ]:
# 프로젝트 이름
proj_name = "Base"

In [ ]:
for i in range(len(df)):
    client.trace(
        project_name=proj_name,
        input={"question": df['question'][i]},
        output={"answer": df['answer'][i]},
        metadata={"contexts": df['contexts'][i]},
        feedback_scores=[{"name": "Faithfulness (Ragas)", "value": score['faithfulness'][i]},
                         {"name": "Answer_Relevancy (Ragas)", "value": score['answer_relevancy'][i]}]
    )